In [ ]:
import pandas as pd

import csv
import time
import datetime

from tqdm import tqdm
from datetime import datetime as dt
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

In [ ]:
def crawlerPlayerInfos(playerName, playingChampion, endDate, game):    
    gp = 'FIRST TIME PLAYING'
    wr ='FIRST TIME PLAYING'
    kda = 'FIRST TIME PLAYING'
    firstTimePlaying = False
    chrome_path = r'dependency/chromedriver'
    options = webdriver.ChromeOptions()
    options.headless = True
    options.add_argument('window-size=1400,600')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(executable_path=chrome_path, options=options)

    driver.get("https://oracleselixir.com/player/{}/championPool".format(playerName))

    # Aguardando carregamento da página e exibição do input de Start Date
    wait = WebDriverWait(driver, 5)
    try:
        start_date_input = wait.until(ec.presence_of_element_located(
                (By.XPATH, '//*[@id="start_date"]')), message='Campo "Start Date" era esperado na busca do jogador {} e não foi encontrado.'.format(playerName))
        start_date_input.send_keys('01/01/2021')
    except TimeoutException as ex:
        with open('outputs/incorrect-names.txt', 'a', encoding='utf-8') as f:
            f.write("{} - {}\n".format(playerName, game))
        return []

    # Aguardando carregamento da página e exibição do input de End Date
    start_date_input = wait.until(ec.presence_of_element_located(
            (By.XPATH, '//*[@id="end_date"]')), message='Campo "Start Date" era esperado na busca do jogador {} e não foi encontrado.'.format(playerName))
    start_date_input.send_keys(endDate)

    # Aguardando carregamento da página e exibição dos resultados
    time.sleep(5)
    
    if ( len(driver.find_elements_by_xpath("//*[text()=\"{}\"]/following::div/following::div".format(playingChampion))) != 0):
        # Obtendo valores de GP, W% e KDA
        firstTimePlaying = True
        games_played_element = driver.find_elements_by_xpath("//*[text()=\"{}\"]/following::div/following::div".format(playingChampion))[0]
        win_rate_element = driver.find_elements_by_xpath("//*[text()=\"{}\"]/following::div/following::div/following::div".format(playingChampion))[0]
        kda_element = driver.find_elements_by_xpath("//*[text()=\"{}\"]/following::div/following::div/following::div/following::div/following::div".format(playingChampion))[0]

        gp = games_played_element.text
        wr = win_rate_element.text
        kda = kda_element.text
  

    driver.quit()

    return gp, wr, kda, firstTimePlaying

In [ ]:
df = pd.read_csv("data/mess_dataset_20210515.csv")
games = df.gameid.drop_duplicates()

header = 'game,bluetop,bluetopgp,bluetopwr,bluetopkda,bluejungle,bluejunglegp,bluejunglewr,bluejunglekda,bluemid,bluemidgp,bluemidwr,bluemidkda,blueadc,blueadcgp,blueadcwr,blueadckda,bluesupport,bluesupportgp,bluesupportwr,bluesupportkda,redtop,redtopgp,redtopwr,redtopkda,redjungle,redjunglegp,redjunglewr,redjunglekda,redmid,redmidgp,redmidwr,redmidkda,redadc,redadcgp,redadcwr,redadckda,redsupport,redsupportgp,redsupportwr,redsupportkda,result\n'
with open('outputs/dataset_20210515.csv', mode='a') as dataset2021:
    dataset2021.write(header)
    
def processGames(game):
    
    if isinstance(game, str):

        # GAME DATETIME
        endDate = dt.strptime(df[(df['gameid']==game)].date.values[0], '%Y-%m-%d %H:%M:%S')
        formatedEndDate = "{:02d}/{:02d}/{}".format(endDate.day - 1,endDate.month,endDate.year)

        # BLUE TEAM
        blueTop = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='top')].champion.values[0]
        blueTopPlayer = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='top')].player.values[0]
        if (blueTopPlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(blueTopPlayer, blueTop, formatedEndDate, game)
            if (crawler):
              blueTopGP = crawler[0]
              blueTopWR = crawler[1]
              blueTopKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return

        blueJungle = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='jng')].champion.values[0]
        blueJunglePlayer = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='jng')].player.values[0]
        if (blueJunglePlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(blueJunglePlayer, blueJungle, formatedEndDate, game)
            if (crawler):
              blueJungleGP = crawler[0]
              blueJungleWR = crawler[1]
              blueJungleKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return

        blueMid = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='mid')].champion.values[0]
        blueMidPlayer = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='mid')].player.values[0]
        if (blueMidPlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(blueMidPlayer, blueMid, formatedEndDate, game)
            if (crawler):
              blueMidGP = crawler[0]
              blueMidWR = crawler[1]
              blueMidKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return

        blueCarry = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='bot')].champion.values[0]
        blueCarryPlayer = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='bot')].player.values[0]
        if (blueCarryPlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(blueCarryPlayer, blueCarry, formatedEndDate, game)
            if (crawler):
              blueCarryGP = crawler[0]
              blueCarryWR = crawler[1]
              blueCarryKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return

        blueSupp = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='sup')].champion.values[0]
        blueSuppPlayer = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='sup')].player.values[0]
        if (blueSuppPlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(blueSuppPlayer, blueSupp, formatedEndDate, game)
            if (crawler):
              blueSuppGP = crawler[0]
              blueSuppWR = crawler[1]
              blueSuppKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return

        # RED TEAM
        redTop = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='top')].champion.values[0]
        redTopPlayer = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='top')].player.values[0]
        if (redTopPlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(redTopPlayer, redTop, formatedEndDate, game)
            if (crawler):
              redTopGP = crawler[0]
              redTopWR = crawler[1]
              redTopKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return

        redJungle = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='jng')].champion.values[0]
        redJunglePlayer = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='jng')].player.values[0]
        if (redJunglePlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(redJunglePlayer, redJungle, formatedEndDate, game)
            if (crawler):
              redJungleGP = crawler[0]
              redJungleWR = crawler[1]
              redJungleKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return

        redMid = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='mid')].champion.values[0]
        redMidPlayer = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='mid')].player.values[0]
        if (redMidPlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(redMidPlayer, redMid, formatedEndDate, game)
            if (crawler):
              redMidGP = crawler[0]
              redMidWR = crawler[1]
              redMidKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return

        redCarry = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='bot')].champion.values[0]
        redCarryPlayer = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='bot')].player.values[0]
        if (redCarryPlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(redCarryPlayer, redCarry, formatedEndDate, game)
            if (crawler):
              redCarryGP = crawler[0]
              redCarryWR = crawler[1]
              redCarryKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return
        
        redSupp = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='sup')].champion.values[0]
        redSuppPlayer = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='sup')].player.values[0]
        if (redSuppPlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(redSuppPlayer, redSupp, formatedEndDate, game)
            if (crawler):
              redSuppGP = crawler[0]
              redSuppWR = crawler[1]
              redSuppKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return

        # RESULT
        result = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='sup')].result.values[0]

        # WRITING TO DATASET FILE        
        with open('outputs/dataset_20210515.csv', mode='a', newline="") as dataset2021:
            datasetWriter = csv.writer(dataset2021, delimiter=',')
            datasetWriter.writerow([game,blueTop, blueTopGP, blueTopWR, blueTopKDA, blueJungle, blueJungleGP, blueJungleWR, blueJungleKDA, blueMid, blueMidGP, blueMidWR, blueMidKDA, blueCarry, blueCarryGP, blueCarryWR, blueCarryKDA, blueSupp, blueSuppGP, blueSuppWR, blueSuppKDA, redTop,redTopGP, redTopWR, redTopKDA, redJungle, redJungleGP, redJungleWR, redJungleKDA, redMid, redMidGP, redMidWR, redMidKDA, redCarry, redCarryGP, redCarryWR, redCarryKDA, redSupp, redSuppGP, redSuppWR, redSuppKDA, result])        


In [ ]:
for game in tqdm(games):
    processGames(game)